# Setting

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# Quick Start

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

response = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)
print(response)

content='Your name is Bob. How can I help you today, Bob?' response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-759596cd-9709-42bc-943f-e007c54e9272-0' usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49}


# Message History

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 메시지 히스토리를 저장할 저장소
store = {}

# 세션 ID를 받아서 해당 세션의 메시지 히스토리를 반환하는 함수
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 모델을 감싸고 메시지 히스토리를 추가하는 체인 설정
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

print(response.content)  # 'Hi Bob! How can I assist you today?'

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)  # 'Your name is Bob. How can I help you today, Bob?'

Hello Bob! How can I assist you today?
Your name is Bob. How can I assist you today, Bob?


In [14]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)  # "I'm sorry, I cannot determine your name as I am an AI assistant and do not have access to that information."

I'm sorry, I do not have the capability to know your name.


In [15]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)  # 'Your name is Bob. How can I assist you today, Bob?'

Your name is Bob. How can I assist you today, Bob?


# Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 시스템 메시지와 메시지 플레이스홀더를 포함한 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# 모델과 프롬프트 템플릿을 체인으로 연결
chain = prompt | model

In [17]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
print(response.content)  # 'Hello Bob! How can I assist you today?'

Hello, Bob! How can I assist you today?


In [24]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [25]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim. How can I assist you further, Jim?'

## 언어 자유 설정 예

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Korean"}
)

response.content

'안녕하세요, 밥님! 어떻게 도와드릴까요?'

## prompt & model - chain 심화

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [29]:
config = {"configurable": {"session_id": "abc11"}}

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Korean"},
    config=config,
)

response.content

'안녕하세요, 토드님! 만나서 반가워요. 어떻게 도와드릴까요?'

In [31]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Korean"},
    config=config,
)

response.content

'당신의 이름은 토드입니다.'

# Managing Conversation History

### Trimmer

messages가 마치 vector space 느낌의 예제

In [32]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [33]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to personal information. How can I assist you today?"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what is 2 + 2.'

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [36]:
# 이름 물어보기 예
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information."

In [38]:
# 수학 문제 물어보기 예
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "what\'s 2 + 2?"'

# Streaming

보다 나은 사용자 경험을 위한 로딩 상황 시각화
* 토큰 단위로 바로바로 보여줌

In [39]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Sure|,| here|'s| a| joke| for| you|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!||